In [26]:
import os
import pandas as pd


In [47]:
base_path = r'C:\Users\zico2m\Desktop\projectzico'

In [48]:
city_attributes = pd.read_csv(f'{base_path}\city_attributes.csv')
humidity_chunksize = 10000  # حجم الشريحة الذي سيتم تحميله في الذاكرة
pressure_chunksize = 10000  # حجم الشريحة لملف الضغط
weather_description_chunksize = 10000  # حجم الشريحة لوصف الطقس
temperature_chunksize = 10000  # حجم الشريحة لملف درجة الحرارة
wind_direction_chunksize = 10000  # حجم الشريحة لملف اتجاه الرياح


In [49]:
humidity_chunks = pd.read_csv(f'{base_path}/humidity.csv', chunksize=humidity_chunksize)
pressure_chunks = pd.read_csv(f'{base_path}/pressure.csv', chunksize=pressure_chunksize)
weather_description_chunks = pd.read_csv(f'{base_path}/weather_description.csv', chunksize=weather_description_chunksize)
temperature_chunks = pd.read_csv(f'{base_path}/temperature.csv', chunksize=temperature_chunksize)
wind_direction_chunks = pd.read_csv(f'{base_path}/wind_direction.csv', chunksize=wind_direction_chunksize)

In [50]:
df = city_attributes.head()
df

,City,Country,Latitude,Longitude
0,Vancouver,Canada,49.249660,-123.119339
1,Portland,United States,45.523449,-122.676208
2,San Francisco,United States,37.774929,-122.419418
3,Seattle,United States,47.606209,-122.332069
4,Los Angeles,United States,34.052231,-118.243683


In [51]:
df = pd.DataFrame()

# معالجة كل كتلة من البيانات بشكل منفصل
for chunk in humidity_chunks:
    chunk = chunk.melt(id_vars=['datetime'], var_name='City', value_name='Humidity')
    df = pd.concat([df, chunk], ignore_index=True)

for chunk in pressure_chunks:
    chunk = chunk.melt(id_vars=['datetime'], var_name='City', value_name='Pressure')
    df = pd.concat([df, chunk], ignore_index=True)

for chunk in weather_description_chunks:
    chunk = chunk.melt(id_vars=['datetime'], var_name='City', value_name='Weather_Description')
    df = pd.concat([df, chunk], ignore_index=True)

for chunk in temperature_chunks:
    chunk = chunk.melt(id_vars=['datetime'], var_name='City', value_name='Temperature')
    df = pd.concat([df, chunk], ignore_index=True)

for chunk in wind_direction_chunks:
    chunk = chunk.melt(id_vars=['datetime'], var_name='City', value_name='Wind_Direction')
    df = pd.concat([df, chunk], ignore_index=True)

# تحويل العمود 'datetime' إلى نوع البيانات المناسب
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# تحويل الأعمدة 'City' إلى نوع 'category' لتقليل استخدام الذاكرة
df['City'] = df['City'].astype('category')

# حفظ البيانات المدمجة في ملف CSV مؤقت
df.to_csv(f'{base_path}/merged_data.csv', index=False)

# عرض أول 5 صفوف من البيانات المدمجة
print(df.head())

             datetime       City  Humidity  Pressure Weather_Description  \
0 2012-10-01 12:00:00  Vancouver       NaN       NaN                 NaN   
1 2012-10-01 13:00:00  Vancouver      76.0       NaN                 NaN   
2 2012-10-01 14:00:00  Vancouver      76.0       NaN                 NaN   
3 2012-10-01 15:00:00  Vancouver      76.0       NaN                 NaN   
4 2012-10-01 16:00:00  Vancouver      77.0       NaN                 NaN   

   Temperature  Wind_Direction  
0          NaN             NaN  
1          NaN             NaN  
2          NaN             NaN  
3          NaN             NaN  
4          NaN             NaN  


In [52]:
# فحص البيانات المدمجة
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145540 entries, 0 to 8145539
Data columns (total 7 columns):
 #   Column               Dtype         
---  ------               -----         
 0   datetime             datetime64[ns]
 1   City                 category      
 2   Humidity             float64       
 3   Pressure             float64       
 4   Weather_Description  object        
 5   Temperature          float64       
 6   Wind_Direction       float64       
dtypes: category(1), datetime64[ns](1), float64(4), object(1)
memory usage: 380.6+ MB
None
             datetime       City  Humidity  Pressure Weather_Description  \
0 2012-10-01 12:00:00  Vancouver       NaN       NaN                 NaN   
1 2012-10-01 13:00:00  Vancouver      76.0       NaN                 NaN   
2 2012-10-01 14:00:00  Vancouver      76.0       NaN                 NaN   
3 2012-10-01 15:00:00  Vancouver      76.0       NaN                 NaN   
4 2012-10-01 16:00:00  Vancouver      77.0      

In [53]:
# التحقق من القيم المفقودة
print(df.isnull().sum())


datetime                     0
City                         0
Humidity               6545083
Pressure               6533112
Weather_Description    6524387
Temperature            6524462
Wind_Direction         6524407
dtype: int64


In [54]:
df.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم مفقودة


In [55]:
print(df.describe())


       Humidity  Pressure  Temperature  Wind_Direction
count       0.0       0.0          0.0             0.0
mean        NaN       NaN          NaN             NaN
std         NaN       NaN          NaN             NaN
min         NaN       NaN          NaN             NaN
25%         NaN       NaN          NaN             NaN
50%         NaN       NaN          NaN             NaN
75%         NaN       NaN          NaN             NaN
max         NaN       NaN          NaN             NaN


In [ ]:
print(df.columns)


Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa',
       'Nahariyya', 'Jerusalem'],
      dtype='object')
